In [7]:
import queue  
from collections import namedtuple

Edge = namedtuple('Edge', ['vertex', 'weight'])


class GraphUndirectedWeighted(object):  
    def __init__(self, vertex_count):
        self.vertex_count = vertex_count
        self.adjacency_list = [[] for _ in range(vertex_count)]

    def add_edge(self, source, dest, weight):
        assert source < self.vertex_count
        assert dest < self.vertex_count
        self.adjacency_list[source].append(Edge(dest, weight))
        self.adjacency_list[dest].append(Edge(source, weight))

    def get_edge(self, vertex):
        for e in self.adjacency_list[vertex]:
            yield e

    def get_vertex(self):
        for v in range(self.vertex_count):
            yield v


def dijkstra(graph, source, dest):  
    q = queue.PriorityQueue()
    parents = []
    distances = []
    start_weight = float("inf")

    for i in graph.get_vertex():
        weight = start_weight
        if source == i:
            weight = 0
        distances.append(weight)
        parents.append(None)

    q.put(([0, source]))

    while not q.empty():
        v_tuple = q.get()
        v = v_tuple[1]

        for e in graph.get_edge(v):
            candidate_distance = distances[v] + e.weight
            if distances[e.vertex] > candidate_distance:
                distances[e.vertex] = candidate_distance
                parents[e.vertex] = v
                # primitive but effective negative cycle detection
                if candidate_distance < -1000:
                    raise Exception("Negative cycle detected")
                q.put(([distances[e.vertex], e.vertex]))

    shortest_path = []
    end = dest
    while end is not None:
        shortest_path.append(end)
        end = parents[end]

    shortest_path.reverse()

    return shortest_path, distances[dest]


def main():  
    g = GraphUndirectedWeighted(9)
    g.add_edge(0, 1, 4)
    g.add_edge(1, 7, 6)
    g.add_edge(1, 2, 1)
    g.add_edge(2, 3, 3)
    g.add_edge(3, 7, 1)
    g.add_edge(3, 4, 2)
    g.add_edge(3, 5, 1)
    g.add_edge(4, 5, 1)
    g.add_edge(5, 6, 1)
    g.add_edge(6, 7, 2)
    g.add_edge(6, 8, 2)
    g.add_edge(7, 8, 2)
    # for testing negative cycles
    # g.add_edge(1, 9, -5)
    # g.add_edge(9, 7, -4)

    shortest_path, distance = dijkstra(g, 0, 1)
    assert shortest_path == [0, 1] and distance == 4

    shortest_path, distance = dijkstra(g, 0, 8)
    assert shortest_path == [0, 1, 2, 3, 7, 8] and distance == 11

    shortest_path, distance = dijkstra(g, 5, 0)
    assert shortest_path == [5, 3, 2, 1, 0] and distance == 9
    
    print(distance)

import numpy as np
import scipy.misc
import scipy as sp
import time

def main2():
    row = 20
    col = 20
    length = row * col
    matrix = scipy.misc.imread('../image/mountain2.jpg', mode="L")

    matrix = sp.misc.imresize(matrix, (row, col)) 

    # print(matrix)
    # print(matrix.shape)
    
    g = GraphUndirectedWeighted(length)

    for i in range(row):
        for j in range(col):
            if 0 < i < col-1 and 0 < j < row-1:
                edge_value1 = abs(matrix[i, j] - matrix[i-1, j])
                g.add_edge(i*col+j, (i-1)*col+j, edge_value1)
                
                edge_value2 = abs(matrix[i, j] - matrix[i, j-1])
                g.add_edge(i*col+j, i*col+j-1, edge_value2)
                
                edge_value3 = abs(matrix[i, j] - matrix[i+1, j])
                g.add_edge(i*col+j, (i+1)*col+j, edge_value3)
                
                edge_value4 = abs(matrix[i, j] - matrix[i, j+1])
                g.add_edge(i*col+j, i*col+j+1, edge_value4)
                
            elif 0 < i < row-1 and (j == 0 or j == row-1):
                edge_value5 = abs(matrix[i, j] - matrix[i-1, j])
                g.add_edge(i*col+j, (i-1)*col+j, edge_value5)
                
                edge_value6 = abs(matrix[i, j] - matrix[i+1, j])
                g.add_edge(i*col+j, (i+1)*col+j, edge_value6)
            elif 0 < j < col-1 and (i == 0 or i == col-1):
                edge_value7 = abs(matrix[i, j] - matrix[i, j-1])
                g.add_edge(i*col+j, i*col+j-1, edge_value7)
                
                edge_value8 = abs(matrix[i, j] - matrix[i, j+1])
                g.add_edge(i*col+j, i*col+j+1, edge_value8)
            elif i == 0 and j == 0:
                g.add_edge(0, 1, abs(matrix[0, 0]-matrix[0, 1]))
                g.add_edge(0, col, abs(matrix[0, 0]-matrix[1, 0]))
            elif i == 0 and j == row-1:
                g.add_edge(row-1, row-2, abs(matrix[0, row-1]-matrix[0, row-2]))
                g.add_edge(row-1, col+row-1, abs(matrix[0, row-1]-matrix[1, row-1]))
            elif i == col-1 and j == 0:
                g.add_edge(i*row, (i-1)*row, abs(matrix[i, 0]-matrix[row-1, 0]-matrix[row-2, 0]))
                g.add_edge(i*row, i*row+1, abs(matrix[row-1, 0]-matrix[row-1, 1]))
            elif i == col-1 and j == row-1:
                g.add_edge(i*row+j, i*row+j-1, abs(matrix[row-1, col-1]-matrix[row-1, col-2]))
                g.add_edge(i*row+j, (i-1)*row+j, abs(matrix[row-1, col-1]-matrix[row-2, col-1]))
                
            
    shortest_path, distance = dijkstra(g, 20, 20)
    print(distance)

    shortest_path, distance = dijkstra(g, 10, 20)
    print(distance)
    
    start_time = time.time()
    
    distanceMatrix = []
    for i in range(length):
        for j in range(length):
            shortest_path, distance = dijkstra(g, i, j)
            distanceMatrix.append(distance)
            
    distanceMatrix = np.reshape(distanceMatrix, (length, length))
    
    print(distanceMatrix)
    print("distance matrix construct time: ", time.time()-start_time)
    return distanceMatrix
    
    
if __name__ == "__main__":  
    main2()

/home/yy/env/acaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:136: RuntimeWarning: overflow encountered in ubyte_scalars
/home/yy/env/acaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:139: RuntimeWarning: overflow encountered in ubyte_scalars
/home/yy/env/acaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:130: RuntimeWarning: overflow encountered in ubyte_scalars
/home/yy/env/acaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:117: RuntimeWarning: overflow encountered in ubyte_scalars
/home/yy/env/acaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:120: RuntimeWarning: overflow encountered in ubyte_scalars
/home/yy/env/acaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:126: RuntimeWarning: overflow encountered in ubyte_scalars
/home/yy/env/acaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:123: RuntimeWarning: overflow encountered in ubyte_scalars
/home/yy/env/acaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:133

0
43
[[  0   9   9 ..., 342 346 342]
 [  9   0   0 ..., 333 337 333]
 [  9   0   0 ..., 333 337 333]
 ..., 
 [342 333 333 ...,   0   4  20]
 [346 337 337 ...,   4   0  16]
 [342 333 333 ...,  20  16   0]]
distance matrix construct time:  1036.5243618488312
